In [1]:
from huggingface_hub import login
import torch
import os
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder
from dotenv import load_dotenv

d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
token = os.getenv("HF_TOKEN")

login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
embedding_id = "distiluse-base-multilingual-cased-v2"

embeddings = HuggingFaceEmbeddings(model_name=embedding_id, cache_folder="./model_cache")

In [5]:
reranker_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"

reranker = CrossEncoder(model_name=reranker_id, cache_dir="./model_cache")

In [6]:
reranker_2 = HuggingFaceCrossEncoder(model_name=reranker_id, model_kwargs={'cache_dir' : "./model_cache"})

In [ ]:
hf = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Llama-3.2-1B",
    task="text-generation",
    device_map="auto",
    pipeline_kwargs={"max_new_tokens" : 10},
    model_kwargs={
        'cache_dir' : './model_cache', 
        'torch_dtype' : torch.bfloat16,
        'quantization_config' : {"load_in_4bit" : True},
        'low_cpu_mem_usage': True
        }
    )

Device set to use cuda:0


In [5]:
hf("Berikan aku resep nasi goreng")

C:\Users\rizfi\AppData\Local\Temp\ipykernel_24840\1563240608.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hf("Berikan aku resep nasi goreng")


'Berikan aku resep nasi goreng yang enak. Berikut ini resep n'

In [1]:
import torch
from transformers import pipeline

d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    model_kwargs={'cache_dir' : './model_cache'},
    max_new_tokens=256
)

Device set to use cuda:0


In [6]:
llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is electroencephalography?"

response = chain.invoke({"question": question})

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is electroencephalography?

Answer: Let's think step by step. Electroencephalography (EEG) is a diagnostic tool that records the electrical activity of the brain. This is done using electrodes placed on the scalp to capture the brain's electrical signals. These signals are typically measured in terms of their frequency, amplitude, and synchronization, which can be used to diagnose and monitor various neurological conditions.

Step 1: Identify the main purpose of EEG
The main purpose of EEG is to record the electrical activity of the brain.

Step 2: Consider the types of signals recorded by EEG
EEG records various types of signals, including:
- Brain waves: delta, theta, alpha, beta, and theta-4 wave activity.
- Cortical activity: local field potentials and synchronized neural activity.
- Transient activity: spike and wave discharges.

Step 3: Analyze the applications of EEG
EEG has a wide range of applications, including:
- Diagnosing and monitoring neurological conditio